In [4]:
# print all the folder in the folder named flowers

import os
import pandas as pd
import cv2

# craete an empty pandas dataframe with columns as image, label
df = pd.DataFrame(columns=['image', 'label'])

flower_types=[]

for folder in os.listdir('flowers'):
    flower_types.append(folder)
    print("Curr Folder: ", folder + "\n")
    # read all the images in the folder
    for image in os.listdir('flowers/'+folder):
        # read the image
        img = cv2.imread('flowers/'+folder+'/'+image)
        # resize the image
        img = cv2.resize(img, (64, 64))
        # add the image to dataframe
        df = df.append({'image': img, 'label': folder}, ignore_index=True)







Curr Folder:  daisy

Curr Folder:  dandelion

Curr Folder:  rose

Curr Folder:  sunflower

Curr Folder:  tulip



In [16]:
# split data into test and train and validation
from sklearn.model_selection import train_test_split
import numpy as np

# shuffle the data
df = df.sample(frac=1).reset_index(drop=True)


# split the data into train and test
train, test = train_test_split(df, test_size=0.2, random_state=42)

# split the train data into train and validation
train, validation = train_test_split(train, test_size=0.2, random_state=42)

# print the shape of train, test and validation data
print("Train shape: ", train.shape)
print("Test shape: ", test.shape)
print("Validation shape: ", validation.shape)

# convert the train, test and validation data into numpy array
train_x = np.array(train['image'].tolist())
train_y = np.array(train['label'].tolist())

test_x = np.array(test['image'].tolist())
test_y = np.array(test['label'].tolist())

validation_x = np.array(validation['image'].tolist())
validation_y = np.array(validation['label'].tolist())

# replace the labels with numbers according to the flower_types list
for i in range(len(flower_types)):
    train_y[train_y == flower_types[i]] = i
    test_y[test_y == flower_types[i]] = i
    validation_y[validation_y == flower_types[i]] = i



Train shape:  (2762, 2)
Test shape:  (864, 2)
Validation shape:  (691, 2)


In [17]:
# create a cnn model for classification
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU

# create a cnn model
input_shape = (64, 64, 3)
num_classes = len(flower_types)

model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=input_shape))
model.add(Conv2D(32, kernel_size=3, activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(16, kernel_size=3, activation="relu"))
model.add(Conv2D(8, kernel_size=3, activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(num_classes, activation="softmax"))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model
model.fit(train_x, to_categorical(train_y), epochs=20, validation_data=(validation_x, to_categorical(validation_y)))

# evaluate the model
model.evaluate(test_x, to_categorical(test_y))


Epoch 1/20
 9/87 [==>...........................] - ETA: 20s - loss: 2.1318 - accuracy: 0.2708

KeyboardInterrupt: 

In [7]:
# save the model with name image_classification_model.h5
model.save('image_classification_model.h5')


In [20]:
# load flowers\daisy\5547758_eea9edfd54_n.jpg
import cv2
import numpy as np
from keras.models import load_model

model = load_model('image_classification_model.h5')
img = cv2.imread('flowers\daisy\\5547758_eea9edfd54_n.jpg')
img = cv2.resize(img, (64, 64))
img = np.reshape(img, [1, 64, 64, 3])

# print calss from the flower_types list
print("Predicted class: ", flower_types[np.argmax(model.predict(img))])

1/1 [==============================] - 0s 83ms/step
Predicted class:  daisy
